In [1]:
!pip install pandas folium scikit-learn geopy matplotlib seaborn


In [2]:
import pandas as pd


In [4]:
df = pd.read_excel('amazon_delivery.xlsx', engine='openpyxl')
df.head()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys


In [5]:
df.rename(columns={
    'Store_Latitude': 'pickup_lat',
    'Store_Longitude': 'pickup_lng',
    'Drop_Latitude': 'drop_lat',
    'Drop_Longitude': 'drop_lng',
    'Pickup_time': 'pickup_time',
    'Order_Date': 'order_date',
    'Delivery_Time': 'delivery_time_mins'
}, inplace=True)
df.head()


,Order_ID,Agent_Age,Agent_Rating,pickup_lat,pickup_lng,drop_lat,drop_lng,order_date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,delivery_time_mins,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys


In [6]:
from datetime import timedelta
df['pickup_datetime']=pd.to_datetime(df['order_date'].astype(str)+' '+df['Pickup_Time'].astype(str) )
df['actual_delivery_time'] = df['pickup_datetime'] + pd.to_timedelta(df['delivery_time_mins'], unit='m')
df['expected_delivery_time'] = df['pickup_datetime'] + timedelta(minutes=30)
df['delay_minutes'] = (df['actual_delivery_time'] - df['expected_delivery_time']).dt.total_seconds() / 60
df['is_delayed'] = df['delay_minutes'] > 0
df[['Order_ID', 'pickup_datetime', 'actual_delivery_time', 'delay_minutes', 'is_delayed']].head()

,Order_ID,pickup_datetime,actual_delivery_time,delay_minutes,is_delayed
0,ialx566343618,2022-03-19 11:45:00,2022-03-19 13:45:00,90.0,True
1,akqg208421122,2022-03-25 19:50:00,2022-03-25 22:35:00,135.0,True
2,njpu434582536,2022-03-19 08:45:00,2022-03-19 10:55:00,100.0,True
3,rjto796129700,2022-04-05 18:10:00,2022-04-05 19:55:00,75.0,True
4,zguw716275638,2022-03-26 13:45:00,2022-03-26 16:15:00,120.0,True


In [7]:
import folium
from folium.plugins import MarkerCluster

map_center = [df['drop_lat'].mean(), df['drop_lng'].mean()]
m = folium.Map(location=map_center, zoom_start=11)

cluster = MarkerCluster().add_to(m)

for _, row in df[df['is_delayed']].iterrows():
    folium.Marker(
        location=[row['drop_lat'], row['drop_lng']],
        popup=(
            f"Order ID: {row['Order_ID']}<br>"
            f"Delay: {int(row['delay_minutes'])} min<br>"
            f"Vehicle: {row['Vehicle']}<br>"
            f"Area: {row['Area']}<br>"
            f"Weather: {row['Weather']}<br>"
            f"Traffic: {row['Traffic']}"
        ),
        icon=folium.Icon(color='red', icon='exclamation-sign')
    ).add_to(cluster)

m

Output hidden; open in https://colab.research.google.com to view.

In [8]:
m.save("amazon_delivery_delay_map.html")

In [9]:

!pip install folium branca

In [10]:
import folium
from folium.plugins import MarkerCluster, Fullscreen

def delay_color(delay):
    if delay < 30:
        return 'orange'
    elif delay < 60:
        return 'darkred'
    else:
        return 'purple'

map_center = [df['drop_lat'].mean(), df['drop_lng'].mean()]
m = folium.Map(location=map_center, zoom_start=11, tiles='CartoDB positron')

Fullscreen().add_to(m)

cluster = MarkerCluster().add_to(m)

for _, row in df[df['is_delayed']].iterrows():
    delay = int(row['delay_minutes'])
    folium.Marker(
        location=[row['drop_lat'], row['drop_lng']],
        popup=folium.Popup(
            f"""
            <b>Order ID:</b> {row['Order_ID']}<br>
            <b>Delay:</b> {delay} min<br>
            <b>Area:</b> {row['Area']}<br>
            <b>Traffic:</b> {row['Traffic']}<br>
            <b>Weather:</b> {row['Weather']}<br>
            <b>Vehicle:</b> {row['Vehicle']}
            """, max_width=250),
        tooltip=f"Delay: {delay} min",
        icon=folium.Icon(color=delay_color(delay), icon="clock", prefix="fa")
    ).add_to(cluster)

m

Output hidden; open in https://colab.research.google.com to view.

In [11]:
from branca.element import Template, MacroElement
import folium

legend_html = """
<div style="
    position: fixed;
    bottom: 50px; left: 50px; width: 160px; height: 110px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
    ">
    <b>Delay Legend</b><br>
    <i class="fa fa-map-marker fa-2x" style="color:orange"></i>&nbsp; < 30 min<br>
    <i class="fa fa-map-marker fa-2x" style="color:darkred"></i>&nbsp; 30–60 min<br>
    <i class="fa fa-map-marker fa-2x" style="color:purple"></i>&nbsp; > 60 min
</div>
"""

legend = folium.Element(legend_html)
m.get_root().add_child(legend)

m

Output hidden; open in https://colab.research.google.com to view.

In [12]:
from folium import Element

legend_html = """
<div style='position: fixed;
     bottom: 50px; left: 50px; width: 180px; height: 115px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; padding: 10px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3);'>
  <b>Delay Legend</b><br>
  <i style="background-color:orange; width: 12px; height: 12px; display: inline-block;"></i> &lt; 30 min<br>
  <i style="background-color:darkred; width: 12px; height: 12px; display: inline-block;"></i> 30–60 min<br>
  <i style="background-color:purple; width: 12px; height: 12px; display: inline-block;"></i> &gt; 60 min
</div>
"""

m.get_root().html.add_child(Element(legend_html))

In [13]:
m

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
m.save("amazon_delivery_delay_legend_map.html")

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

locations = df[['drop_lat', 'drop_lng']]
kmeans = KMeans(n_clusters=8, random_state=42)
df['cluster'] = kmeans.fit_predict(locations)

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['drop_lng'], df['drop_lat'], c=df['cluster'], cmap='tab10', alpha=0.7)
plt.title("Optimized Delivery Zones (KMeans Clustering)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True)
plt.show()

In [ ]:
import seaborn as sns

# Delay by cluster
plt.figure(figsize=(8, 5))
sns.barplot(x='cluster', y='delay_minutes', data=df[df['is_delayed']])
plt.title("Average Delay per Cluster")
plt.ylabel("Avg Delay (min)")
plt.xlabel("Cluster ID")
plt.show()

In [ ]:
from geopy.distance import geodesic

def route_distance(group):
    points = group[['drop_lat', 'drop_lng']].values
    return sum(geodesic(points[i], points[i+1]).km for i in range(len(points)-1))

total_distance = df.groupby('cluster').apply(route_distance).sum()

total_distance = df.groupby('cluster').apply(route_distance).sum()
print(f"Estimated optimized route distance: {total_distance:.2f} km")

In [ ]:
df.to_csv("final_delivery_clusters.csv", index=False)

from google.colab import files
files.download("final_delivery_clusters.csv")

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(x='cluster', y='delay_minutes', data=df[df['is_delayed']])
plt.title("Avg Delay per Cluster")
plt.xlabel("Cluster")
plt.ylabel("Delay (min)")
plt.savefig("avg_delay_per_cluster.png")  # Saves as image

files.download("avg_delay_per_cluster.png")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(df['drop_lng'], df['drop_lat'], c=df['cluster'], cmap='tab10', alpha=0.7)
plt.title('Optimized Delivery Zones (KMeans Clustering)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)

# Save as PNG
plt.savefig("delivery_zones_clustering.png")

plt.show()